# הסתברויות נפוצות ב־NumPy והיסטוגרמות — מדריך קצר

מטרת הפרק: להכיר דגימה מהתפלגויות הסתברות נפוצות באמצעות `numpy.random.Generator`,
לבנות אינטואיציה עם היסטוגרמות, ולהדגים עקרונות יסוד כמו **חוק המספרים הגדולים** ו־**משפט הגבול המרכזי**.
רמת ההסבר מותאמת לבוגר/ת תיכון (עם חיזוקים מתמטיים קלים כשצריך).

## הגדרות בסיסיות והכנות

נשתמש ב־NumPy ליצירת דגימות ובהצגה גרפית עם Matplotlib. נגדיר מחולל אקראיות
(reproducibility) כדי שנקבל תוצאות שחוזרות על עצמן בהרצה חוזרת.


In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# מחולל אקראיות מודרני של NumPy
rng = np.random.default_rng(42)

# פונקציית עזר: שרטוט היסטוגרמה "נקייה"
def plot_hist(data, bins=30, density=False, title=None, xlabel=None, ylabel=None):
    plt.figure(figsize=(6, 4))
    plt.hist(data, bins=bins, density=density)
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    plt.show()

## ברנולי ובינומי

**ברנולי**: ניסוי עם שתי תוצאות \- "הצלחה" (1) בהסתברות \\(p\\) או "כישלון" (0) בהסתברות \\(1-p\\).  
**בינומי**: מספר ההצלחות מתוך \\(n\\) ניסויי ברנולי בלתי\-תלויים עם אותה \\(p\\).

דוגמאות: הטלת מטבע מוטה (\\(p\\) לקבל "עץ"), בדיקת הצלחה/כישלון בניסוי מעבדה, ועוד.
נטעם מספרים ונראה את התוצאות אמפירית.


In [ ]:
# ברנולי עם p=0.3
p = 0.3
N = 10000
ber = rng.binomial(n=1, p=p, size=N)  # או rng.random(N) < p

print("שיעור הצלחות אמפירי:", ber.mean())
plot_hist(ber, bins=[-0.5, 0.5, 1.5], density=True, 
          title="ברנולי p=0.3 — היסטוגרמה (PMF אמפירית)",
          xlabel="תוצאה (0/1)", ylabel="שכיחות יחסית")


In [ ]:
# בינומי: n ניסויים עצמאיים, p אותה הסתברות בכל ניסוי
n = 10
p = 0.3
N = 20000
binom = rng.binomial(n=n, p=p, size=N)

print("תוחלת תיאורטית (n*p):", n*p, "  |  אמפירית:", binom.mean())
# היסטוגרמה עם סלילים המתאימים לערכים שלמים (0..n)
bins = np.arange(-0.5, n + 1.5, 1)
plot_hist(binom, bins=bins, density=True, 
          title=f"בינומי(n={n}, p={p}) — היסטוגרמה (PMF אמפירית)",
          xlabel="מספר הצלחות ב- n ניסויים", ylabel="שכיחות יחסית")


## גאומטרית (Geometric)

ההתפלגות הגאומטרית מתארת את **מספר הניסיונות עד להצלחה הראשונה** כשבכל ניסיון
ההצלחה בהסתברות \\(p\\). התוחלת היא \\(E[X]=1/p\\).

In [ ]:
p = 0.25
N = 20000
geom = rng.geometric(p, size=N)  # מספר ניסיונות עד להצלחה הראשונה (כולל)

print("תוחלת תיאורטית 1/p =", 1/p, "  |  אמפירית:", geom.mean())
# נשרטט היסטוגרמה עד ערך עליון סביר (לצורך הצגה נוחה)
upper = 15
bins = np.arange(0.5, upper + 1.5, 1)
plot_hist(geom[geom <= upper], bins=bins, density=True,
          title=f"גאומטרית(p={p}) — PMF אמפירית (חתוכה עד {upper})",
          xlabel="מספר ניסיונות עד הצלחה", ylabel="שכיחות יחסית")


## פואסון (Poisson)

התפלגות **פואסון** מתארת מספר מאורעות בפרק זמן/מרחב נתון כאשר המאורעות קורים בקצב ממוצע \\(\lambda\\) ובעצמאות.
הקשר לבינומי: כאשר \\(n\\) גדול ו־\\(p\\) קטן כך ש־\\(np = \lambda\\) קבוע, הבינומי מתכנס לפואסון.

ה־PMF (פונקציית מסה) של פואסון: \\(P(X=k)=e^{-\lambda}\lambda^k/k!\\).

In [ ]:
lam = 3.5
N = 30000
poi = rng.poisson(lam=lam, size=N)

print("תוחלת תיאורטית λ =", lam, "  |  אמפירית:", poi.mean())

# היסטוגרמה דיסקרטית + השוואה ל-PMF תיאורטית
k_max = 15
bins = np.arange(-0.5, k_max + 1.5, 1)
plt.figure(figsize=(6,4))
plt.hist(poi, bins=bins, density=True)
ks = np.arange(0, k_max + 1)
pmf = np.exp(-lam) * lam**ks / np.array([math.factorial(k) for k in ks])
plt.plot(ks, pmf)  # קו רציף על נקודות ה-PMF (ברירת מחדל של סגנון)
plt.title(f"פואסון(λ={lam}) — PMF אמפירית ותיאורטית")
plt.xlabel("k (מספר מאורעות)")
plt.ylabel("שכיחות יחסית / הסתברות")
plt.show()


## רציפות: אחידה, נורמלית ואקספוננציאלית

### אחידה רציפה \\(\mathcal{U}(a,b)\\)
כל ערך ב\-\\([a,b]\\) שווה\-הסתברות. ה\-PDF הוא קבוע: \\(f(x)=1/(b-a)\\) בתחום, ואפס מחוץ לו.


In [ ]:
a, b = -2, 3
N = 20000
u = rng.uniform(a, b, size=N)

# PDF תיאורטי
xs = np.linspace(a, b, 200)
pdf = np.ones_like(xs) / (b - a)

plt.figure(figsize=(6,4))
plt.hist(u, bins=30, density=True)
plt.plot(xs, pdf)
plt.title(f"אחידה רציפה U({a},{b}) — היסטוגרמה ו-PDF תיאורטי")
plt.xlabel("x")
plt.ylabel("צפיפות")
plt.show()


### נורמלית \\(\mathcal{N}(\mu,\sigma^2)\\)
התפלגות "פעמון" קלאסית. PDF:  
\\[
f(x)=\frac{1}{\sigma\sqrt{2\pi}}\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right).
\\]

In [ ]:
mu, sigma = 0.0, 1.0
N = 40000
normal = rng.normal(mu, sigma, size=N)

xs = np.linspace(mu - 4*sigma, mu + 4*sigma, 400)
pdf = (1.0/(sigma*np.sqrt(2*np.pi))) * np.exp(-0.5*((xs-mu)/sigma)**2)

plt.figure(figsize=(6,4))
plt.hist(normal, bins=40, density=True)
plt.plot(xs, pdf)
plt.title(f"נורמלית N({mu},{sigma**2}) — היסטוגרמה ו-PDF תיאורטי")
plt.xlabel("x")
plt.ylabel("צפיפות")
plt.show()


### אקספוננציאלית (Exponential)
מתארת זמן בין מאורעות בקצב \\(\lambda\\) קבוע. PDF: \\(f(x)=\lambda e^{-\lambda x}\\) עבור \\(x\ge 0\\).

In [ ]:
lam = 0.5  # קצב ממוצע לאירוע
N = 30000
exp_samp = rng.exponential(1/lam, size=N)  # NumPy מקבל scale=1/λ

xs = np.linspace(0, np.quantile(exp_samp, 0.99), 400)
pdf = lam * np.exp(-lam*xs)

plt.figure(figsize=(6,4))
plt.hist(exp_samp, bins=40, density=True)
plt.plot(xs, pdf)
plt.title(f"אקספוננציאלית (λ={lam}) — היסטוגרמה ו-PDF תיאורטי")
plt.xlabel("x")
plt.ylabel("צפיפות")
plt.show()


## חוק המספרים הגדולים (LLN)

כאשר מחשבים ממוצע של יותר ויותר דגימות בלתי\-תלויות מאותה התפלגות עם תוחלת סופית,
הממוצע האמפירי מתקרב לתוחלת האמיתית. נמחיש עם ברנולי \\(p=0.3\\).

In [ ]:
p = 0.3
N = 20000
ber = rng.binomial(1, p, size=N)
running_mean = np.cumsum(ber) / np.arange(1, N+1)

plt.figure(figsize=(6,4))
plt.plot(running_mean)
plt.axhline(p)  # תוחלת אמיתית
plt.title("חוק המספרים הגדולים — ממוצע נע מתקרב לתוחלת (p=0.3)")
plt.xlabel("מספר דגימות")
plt.ylabel("ממוצע אמפירי")
plt.show()


## משפט הגבול המרכזי (CLT) — הדגמה פשוטה

כאשר מחשבים ממוצעים של **קבוצות** דגימות בלתי\-תלויות מאותה התפלגות (עם שונות סופית),
התפלגות הממוצע נוטה להיות נורמלית ככל שגודל הקבוצה \\(k\\) גדל. נמחיש עם אקספוננציאלית (שאינה נורמלית).

In [ ]:
lam = 1.0
k = 10           # גודל קבוצה לממוצע
M = 20000        # כמה ממוצעים לחשב
samp = rng.exponential(1/lam, size=(M, k))  # נשאב M קבוצות בגודל k
means = samp.mean(axis=1)

# תיאור נורמלי מקורב: ממוצע = 1/λ, שונות = 1/(k*λ^2)
mu = 1/lam
sigma = 1/(lam*np.sqrt(k))

xs = np.linspace(mu - 4*sigma, mu + 4*sigma, 400)
pdf = (1.0/(sigma*np.sqrt(2*np.pi))) * np.exp(-0.5*((xs-mu)/sigma)**2)

plt.figure(figsize=(6,4))
plt.hist(means, bins=40, density=True)
plt.plot(xs, pdf)
plt.title(f"CLT: ממוצע של {k} אקספוננציאליות — היסטוגרמה וגל פעמון מקורב")
plt.xlabel("ממוצע קבוצה")
plt.ylabel("צפיפות")
plt.show()


## היסטוגרמות — מה, למה ואיך

**היסטוגרמה** מחלקת את ציר \\(x\\) ל־*סלילים* (bins) וסופרת כמה דגימות נופלות בכל סליל.
כאשר `density=True`, סך השטח מתחת לעמודות מנורמל ל־1: זהו קירוב ל־PDF (בהתפלגויות רציפות).

### בחירת מספר סלילים
אין כלל אחד "נכון", אבל כללי אצבע שימושיים:
- \\(\sqrt{N}\\) (שורש מספר הדגימות).
- **Sturges**: \\(\lceil \log_2(N) + 1 \rceil\\).
- **Freedman–Diaconis**: רוחב סליל \\(h = 2\cdot IQR\cdot N^{-1/3}\\), ואז מספר סלילים \\((\max-\min)/h\\).

נדגים אלגוריתם Freedman–Diaconis בפועל.


In [ ]:
# דוגמה עם התפלגות נורמלית
data = rng.normal(0, 1, size=6000)

q25, q75 = np.quantile(data, [0.25, 0.75])
iqr = q75 - q25
n = data.size
h = 2 * iqr * (n ** (-1/3))
if h == 0:
    bins = 30  # מקרה קצה
else:
    bins = int(np.ceil((data.max() - data.min()) / h))

plot_hist(data, bins=bins, density=True,
          title=f"היסטוגרמה עם Freedman–Diaconis (bins≈{bins})",
          xlabel="x", ylabel="צפיפות")


### `numpy.histogram` לעומת `plt.hist`

- `numpy.histogram` **מחזירה** מערכים: `counts, bin_edges` — לספירה/חישוב נוסף.
- `plt.hist` **מציירת** היסטוגרמה, ויכולה גם להחזיר את אותם נתונים למי שרוצה.

נחשב הסתברות אמפירית בעזרת `numpy.histogram`.


In [ ]:
# דוגמה: הסתברות אמפירית ל- X > 1.5 כאשר X ~ N(0,1) (אמפירי, לא תיאורטי)
data = rng.normal(0, 1, size=100000)
threshold = 1.5

# דרך ישירה:
prob_empirical = np.mean(data > threshold)
print("P(X>1.5) אמפירית (ישירה):", prob_empirical)

# דרך היסטוגרמה: נסכם את הסלילים שהקצה הימני שלהם גדול מ-1.5 ונחלק ב-N
counts, edges = np.histogram(data, bins=60, density=False)
N = data.size
# מרכזי סלילים
centers = 0.5*(edges[:-1] + edges[1:])
mask = centers > threshold
prob_from_hist = counts[mask].sum() / N
print("P(X>1.5) אמפירית (מסלילים):", prob_from_hist)

### פונקציית התפלגות מצטברת (CDF) אמפירית קצרה

ה־CDF של משתנה מקרי רציף הוא \\(F(x)=P(X\\le x)\\). אמפירית אפשר להעריך על ידי מיון והצטברות.


In [ ]:
# CDF אמפירית עבור דגימות נורמליות
data = rng.normal(0, 1, size=20000)
x_eval = np.linspace(-3.5, 3.5, 200)
# לכל x נחושב חלק יחסי של דגימות <= x (יעיל פחות, אבל ברור)
cdf_emp = np.array([(data <= x).mean() for x in x_eval])

plt.figure(figsize=(6,4))
plt.plot(x_eval, cdf_emp)
plt.title("CDF אמפירית — N(0,1)")
plt.xlabel("x")
plt.ylabel("F(x) ≈ P(X≤x)")
plt.show()


## מה לקחת הלאה
- דגימה מהתפלגויות נפוצות באמצעות `rng` מאפשרת סימולציה מהירה ונשלטת.
- היסטוגרמות הן כלי חזותי מרכזי להבנת הצורה של התפלגות ולבדיקת השערות.
- LLN ו־CLT מסבירים למה ממוצעים "מתנהגים יפה" וכיצד התפלגויות מסתכמות/מתמוצעות לכיוון נורמלי.

> טיפ: כאשר בוחרים bins, נסו כמה כללים ובדקו שהמסקנות **לא תלויות** בבחירה פרטנית מידי.
